In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os
import polars as pl

import random
random.seed(42)

import sys

sys.path.append('../')
sys.path.append('../networks')
from dotenv import load_dotenv

import plotly.express as px

# Load the environment variables from the .env file
dotenv_path = os.path.join('../', '.env')
load_dotenv(dotenv_path)

True

In [2]:
from functions.env import DB_SCIENCE_PATH

conn = sqlite3.connect(DB_SCIENCE_PATH)
conn_full_db = sqlite3.connect(os.getenv("FULL_DB_PATH"))

columns_to_keep = [
        "re_arabic_world",
        "re_central_europe",
        "re_chinese_world",
        "re_eastern_europe",
        "re_france",
        "re_german_world",
        "re_greek_world",
        "re_indian_world",
        "re_italy",
        "re_japan",
        "re_low_countries",
        "re_nordic_countries",
        "re_persian_world",
        "re_slav_world",
        "re_spain",
        "re_united_kingdom",
    ]


df_occupation = pd.read_sql("SELECT * FROM individual_id_cleaned_occupations", conn)

df_meta = pd.read_sql("SELECT * FROM temporal_data", conn)
df_meta = df_meta[df_meta['region_code'].isin(columns_to_keep)]

In [13]:
df_gender = pd.read_sql("SELECT * FROM individual_gender", conn_full_db)
df_gender = df_gender.rename(columns = {'individual_wikidata_id':'wikidata_id'})

In [15]:
df_optimal_partition_weighted =  pd.read_sql("SELECT * FROM optimal_partition_weighted ", conn)
df_optimal_partition_weighted = df_optimal_partition_weighted.sort_values('node', ascending=False).reset_index(drop=True)
df_optimal_partition_weighted.columns = ['meta_occupation', 'community']

df = pd.merge(df_meta, df_occupation, on = 'wikidata_id')
df = pd.merge(df, df_gender, on = 'wikidata_id')
df = pd.merge(df, df_optimal_partition_weighted, on = 'meta_occupation')
df = df.groupby(['wikidata_id', 'region_code', 'interest', 'gender'])['meta_occupation'].count().reset_index()
df = df.sort_values('meta_occupation', ascending=False)

In [38]:
id_counts = df['wikidata_id'].value_counts()

df_specialist = df[df['wikidata_id'].map(id_counts) == 1]
#df_specialist = df_specialist[df_specialist['gender']=='female']
df_specialist = df_specialist[df_specialist['interest']=='nature']

In [39]:
df_occupation[df_occupation['wikidata_id']=='Q2959440']

,wikidata_id,meta_occupation
12954,Q2959440,chemist
12955,Q2959440,paleontologist
12956,Q2959440,botanist
12957,Q2959440,anatomist
12958,Q2959440,zoologist


In [40]:
df_specialist.head(30)

,wikidata_id,region_code,interest,gender,meta_occupation
11919,Q2959440,re_france,nature,male,5
17172,Q442937,re_france,nature,male,5
9101,Q214994,re_german_world,nature,male,4
17459,Q453420,re_united_kingdom,nature,male,4
14823,Q361900,re_united_kingdom,nature,male,4
21153,Q60622,re_german_world,nature,male,4
21644,Q61904,re_german_world,nature,male,4
19008,Q55031155,re_german_world,nature,male,4
2725,Q1248394,re_united_kingdom,nature,male,4
19028,Q55038745,re_german_world,nature,male,4
